機械学習(GB,RF)のパラメータファイルを作るコード


In [ ]:
from datetime import datetime
import os
import time


import numpy as np
import csv


import pandas as pd
from pandas import Series, DataFrame

#importとグラフの可視化の設定
import matplotlib.pyplot as plt
%matplotlib inline

#　スタイルは自分の好みで設定
plt.style.use("ggplot")

#高解像度の設定（きれいな図が出力されますがファイルサイズが大きくなります）
# %config InlineBackend.figure_formats = {'png', 'retina'}




In [ ]:
#機械学習パッケージモジュールのインポート
from sklearn.model_selection import train_test_split

# 識別機の保存はJoblibを用いる
import joblib

In [ ]:
#自作のモジュール
#PYS計算用式が入ったモジュール
from pysfunclib import fowler_func as ff

#機械学習の自動化とプロットのモジュール
from autoreglib import gridreg as gs

In [ ]:
"""
モデルのステップとエネルギー範囲を指定
example: 4.2-6.2eV,0.05eV
        --> xl4262_005
        とする。
"""

xl_name_list=['xl5070_010','xl5070_005',
              'xl4070_010','xl4070_005',
              'xl4270_010',
              'xl4262_010','xl4262_005',
              'xl4060_010',
              'xl4265_005','xl4265_010',
              'xl4067_010','xl4067_005',
              'xl5067_010','xl5067_005',
              'xl3462_010','xl3462_005',
             ]

In [ ]:
#作成した識別機を保存するフォルダー
tdatetime = datetime.now()
tstr = 'spys_reg_{}'.format(tdatetime.strftime('%Y%m%d'))
print('Dir name: ', tstr)

# 名前が「日時」のフォルダを作成。既に存在している場合は上書きされる。
os.makedirs(tstr, exist_ok=True)

#カレントディレクトリーに作成します。その中にファイルが入るようにします。
dir_tstr=".//{}//".format(tstr)


In [ ]:
for ii in xl_name_list:
    
    print("Start time: ",datetime.now().strftime('%Y%m%d %H:%M:%S') )
    start_time = time.time()
    
    #-----------make datasets---------------

    start=float(ii[2:4])*0.1
    end=float(ii[4:6])*0.1
    step=float(ii[7:10])*0.01
    print('energy range, step:', ii)
    print('X data list: start, end, step',start, end, step)
    print()
    
    # Xdata making
    x_data=ff.x_lists(start,end,step)
    
    #Nor setting
    nor_list=ff.x_lists(0.1,3,0.1)
    
    #IP setting （Xdataに依存して作成する。3番目の値から最後から-5までの値まで、0.1刻み)
    ip_list=ff.x_lists(x_data[3],x_data[-5],0.1)

    #Temp setting 
    temp_list=[100,300,500,700,900]
    
    #BGsetting 
    bg_list=[0,5,25,50,100]
    
    total_spectra=len(nor_list)*len(ip_list)*len(temp_list)*len(bg_list)
    
    print('x data(energy):{}'.format(x_data))
    print()
    print('nor:{0}, ip:{1}, tp:{2}, bg:{3}, total spectra:{4}'
          .format(len(nor_list),len(ip_list),len(temp_list),len(bg_list),total_spectra))
    print('nor list:{0}\nip list:{1}\ntp list:{2}\nbg list:{3}'.format(nor_list,ip_list,temp_list,bg_list))
    print()
    
    # SPYS spectra and ip（検出器が飽和するため最大カウントで制限している。）
    XL, yL, yL_all=ff.SPYS_data_ris_max_all(xdata=x_data, nor_list=nor_list, ip_list=ip_list,
                                            temp_list=temp_list, bg_list=bg_list, max_count=150, diff_count=7)
    
    print('total:',total_spectra)
    print('Normal(XL):',len(XL),len(yL))


    X=np.array(XL)
    y=np.array(yL)
    print(X.shape,y.shape)
    print()

    #-----------ML----------


    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, train_size=0.8, random_state=0)
    
    
    start_time_gb = time.time()

    #-----------GB----------
#     gbparam= {'max_depth': [1, 2, 3, 4, 5], 'n_estimators': [5, 10, 50, 100, 200], 'learning_rate': [0.01, 0.1, 1]}
    gbparam={'max_depth': [5], 'n_estimators': [200], 'learning_rate': [0.1]}
    gbreg=gs.grid_gradientboosting(X_train, y_train, X_test, y_test,X_name_list=x_data,params=gbparam)
    
    elapsed_time_gb = time.time() - start_time_gb
    print ("GB_elapsed_time:{0} [sec]".format(elapsed_time_gb))
    print("-----")
    print()
    
    #-----------RF----------
    start_time_rf = time.time()
#     rfparam　= {'n_estimators': [5, 10, 50, 100, 200]}
    rfparam = {'n_estimators': [200]}
    rfreg=gs.grid_randomforest(X_train, y_train, X_test, y_test, x_data, params=rfparam)
    
    elapsed_time_rf = time.time() - start_time_rf
    print ("RF_elapsed_time:{0} [sec]".format(elapsed_time_rf))
    print("-----")
    print()
    
    #　Save Filename
    file_name_gb='sgb_{}.pkl'.format(ii)
    file_name_rf='srf_{}.pkl'.format(ii)
    print("Save File Name: ",file_name_gb, file_name_rf)

    
    dir_tstr_gb=".//{}//{}".format(tstr, file_name_gb)    
    dir_tstr_rf=".//{}//{}".format(tstr, file_name_rf)    
    joblib.dump(gbreg,dir_tstr_gb)
    joblib.dump(rfreg,dir_tstr_rf)
    
    elapsed_time = time.time() - start_time
    
    print ("elapsed_time:{0} [sec]".format(elapsed_time))
    print("finished time: ", datetime.now().strftime('%Y%m%d %H:%M:%S') )
    print("---------------------------------------------------------------------")
    print()

In [ ]:
#Paramater file Load Test
sgb_xl3462_010 =joblib.load('spys_reg_20200623/sgb_xl3462_010.pkl')


In [ ]:
sgb_xl3462_010